Constructing Multipoint Adjoint in Julia v1

* Date created: June 13, 2019
* Name: Sultan Aitzhan
* Description: The construction of the multipoint adjoint is encoded and the test functions in the spirit of \textit{Construct_Adjoint_v1.ipynb} are implemented.

# Importing packages and modules

In [1]:
using SymPy
using Distributions
using ApproxFun
import LinearAlgebra

# Global variables

In [2]:
TOL = 1e-05
DIGITS = 3
INFTY = 10

10

# Helper functions
All functions from this section are copy-and-pasted from $\textit{The_Fokas_Method_documentation_v1.ipynb}.$ Thus, the explanations and examples are omitted.

## `check_all(array, condition)`

In [3]:
function check_all(array, condition)
    for x in array
        if !condition(x)
            return false
        end
    end
    return true
end

check_all (generic function with 1 method)

## `set_tol(x, y)`

In [4]:
function set_tol(x::Union{Number, Array}, y::Union{Number, Array}; atol = TOL)
    if isa(x, Number) && isa(y, Number)
       return atol * mean([x y])
    elseif isa(x, Array) && isa(y, Array)
        if size(x) != size(y)
            throw(error("Array dimensions do not match"))
        end
        # Avoid InexactError() when taking norm()
        x = convert(Array{Complex}, x)
        y = convert(Array{Complex}, y)
        return atol * (LinearAlgebra.norm(x) + LinearAlgebra.norm(y))
    else
        throw(error("Invalid input"))
    end
end

set_tol (generic function with 1 method)

## `evaluate(func, a)`

In [5]:
function evaluate(func::Union{Function, SymPy.Sym, Number}, a::Number)
    if isa(func, Function)
        funcA = func(a)
    elseif isa(func, SymPy.Sym) # SymPy.Sym must come before Number because SymPy.Sym will be recognized as Number
        freeSymbols = free_symbols(func)
        if length(freeSymbols) > 1
            throw(error("func should be univariate"))
        elseif length(freeSymbols) == 1
            t = free_symbols(func)[1,1]
            if isa(a, SymPy.Sym) # if x is SymPy.Sym, do not convert result to Number to preserve pretty printing
                funcA = subs(func, t, a)
            else
                funcA = SymPy.N(subs(func, t, a))
            end
        else
            funcA = func
        end
    else # func is Number
        funcA = func
    end
    return funcA
end

evaluate (generic function with 1 method)

## `partition(n)`

In [6]:
function partition(n::Int)
    output = []
    for i = 0:n
        j = n - i
        push!(output, (i,j))
    end
    return output
end

partition (generic function with 1 method)

## `get_deriv(u, k)`

In [7]:
function get_deriv(u::Union{SymPy.Sym, Number}, k::Int)
    if k < 0
        throw(error("Non-negative k required"))
    end
    if isa(u, SymPy.Sym)
        freeSymbols = free_symbols(u)
        if length(freeSymbols) > 1
            throw(error("u should be univariate"))
        elseif length(freeSymbols) == 1
            t = freeSymbols[1]
            y = u
            for i = 1:k
                newY = diff(y, t) # diff is from SymPy
                y = newY
            end
            return y
        else
            if k > 0
                return 0
            else
                return u
            end
        end
    else
        if k > 0
            return 0
        else
            return u
        end
    end
end

get_deriv (generic function with 1 method)

## `add_func(f, g)`

In [8]:
# Function addition (f + g)(x) := f(x) + g(x)
function add_func(f::Union{Number, Function}, g::Union{Number, Function})
    function h(x)
        if isa(f, Number)
            if isa(g, Number)
                return f + g
            else
                return f + g(x)
            end
        elseif isa(f, Function)
            if isa(g, Number)
                return f(x) + g
            else
                return f(x) + g(x)
            end
        end
    end
    return h
end

add_func (generic function with 1 method)

## `mult_func(f, g)`

In [9]:
# Function multiplication (f * g)(x) := f(x) * g(x)
function mult_func(f::Union{Number, Function}, g::Union{Number, Function})
    function h(x)
        if isa(f, Number)
            if isa(g, Number)
                return f * g
            else
                return f * g(x)
            end
        elseif isa(f, Function)
            if isa(g, Number)
                return f(x) * g
            else
                return f(x) * g(x)
            end
        end
    end
    return h
end

mult_func (generic function with 1 method)

## `sym_to_func(sym)`

In [10]:
function sym_to_func(sym::Union{SymPy.Sym, Number})
    try
        freeSymbols = free_symbols(sym)
        if length(freeSymbols) > 1
            throw(error("sym should be univariate"))
        else
            function func(x)
                if length(freeSymbols) == 0
                    result = SymPy.N(sym)
                else
                    result = SymPy.N(subs(sym, freeSymbols[1], x))
                end
                return result
            end
            return func
        end
    catch
        function func(x)
            return sym
        end
        return func
    end
end

sym_to_func (generic function with 1 method)

## `get_polynomial(coeffs)`
Implement a polynomial in the form of Julia function given an array containing coefficients of $x^n, x^{n-1},..., x^2, x, 1.$


In [11]:
function get_polynomial(coeffList::Array)
    polynomial = 0
    n = length(coeffList)-1
    for i in 0:n
        newTerm = t -> coeffList[i+1] * t^(n-i)
        polynomial = add_func(polynomial, newTerm)
    end
    return polynomial
end

get_polynomial (generic function with 1 method)

## `get_polynomialDeriv(coeffs, k)`
Get the $k$-th derivative of a polynomial implemented above.

In [12]:
function get_polynomialDeriv(coeffList::Array, k::Int)
    if k < 0
        throw(error("Only nonnegative degrees are allowed"))
    elseif k == 0
        newCoeffList = coeffList
    else
        for counter = 1:k
            n = length(coeffList)
            newCoeffList = hcat([0],[(n-i)*coeffList[i] for i in 1:(n-1)]')
            coeffList = newCoeffList
        end
    end
    return get_polynomial(newCoeffList)
end

get_polynomialDeriv (generic function with 1 method)

## `prettyRound(x, digits)`

In [13]:
function prettyRound(x::Number; digs::Int = DIGITS)
    if isa(x, Int)
        return x
    elseif isa(x, Real)
        if isa(x, Rational) || isa(x, Irrational) # If x is rational or irrational numbers like e, pi
            return x
        elseif round(abs(x); digits = digs) == floor(abs(x))
            return Int(round(x))
        else
            return round(x; digits = digs)
        end
    elseif isa(x, Complex)
        roundedReal = prettyRound(real(x); digs = digs)
        roundedComplex = prettyRound(imag(x); digs = digs)
        return roundedReal + im*roundedComplex
    else
        return round(x; digits = digs)
    end
end

prettyRound (generic function with 1 method)

## `prettyPrint(x)`

In [14]:
function prettyPrint(x::Union{Number, SymPy.Sym})
    expr = x
    if isa(expr, SymPy.Sym)
        prettyExpr = expr
        for a in sympy.preorder_traversal(expr)
            if length(free_symbols(a)) == 0 && length(a.args) == 0
                if !(a in [sympy.E, PI]) && length(intersect(a.args, [sympy.E, PI])) == 0 # keep the transcendental numbers as symbols
                    prettyA = prettyRound.(SymPy.N(a))
                    prettyExpr = subs(prettyExpr, (a, prettyA))
                end
            end
        end
    else
        prettyExpr = prettyRound.(expr)
        prettyExpr = convert(SymPy.Sym, prettyExpr)
    end
    return prettyExpr
end

prettyPrint (generic function with 1 method)

In [323]:
t = symbols("t")
x = 1//3*t + (sympy.E)^(2t) + PI/2 + 1.00001*im + sympy.sqrt(2) + 1//6
prettyPrint(x)

t    2⋅t   1        π        
─ + ℯ    + ─ + √2 + ─ + 1.0⋅ⅈ
3          6        2        

In [324]:
t = symbols("t")
x = [1//3*t PI; 1//6*t^2+t 1]
prettyPrint.(x)

2×2 Array{Sym,2}:
       t/3  pi
 t^2/6 + t   1

In [325]:
x = [0.0+1.0*im 1.0+0.0*im -1.0+0.0*im 0.0-1.0*im]
prettyPrint.(x)

1×4 Array{Sym,2}:
 I  1  -1  -I

# Structs

In this section, we developed the relevant structs. 

## `StructDefinitionError`
A struct definition error type is the class of all errors in struct definitions.

In [15]:
struct StructDefinitionError <: Exception
    msg::String
end

## `SymLinearDifferentialOperator(symPFunctions, interval, t)`

In [16]:
struct SymLinearDifferentialOperator
    # Entries in the array should be SymPy.Sym or Number. SymPy.Sym seems to be a subtype of Number, i.e., Array{Union{Number,SymPy.Sym}} returns Array{Number}. But specifying symPFunctions as Array{Number,2} gives a MethodError when the entries are Sympy.Sym objects.
    symPFunctions::Array
    interval::Tuple
    t::SymPy.Sym
    SymLinearDifferentialOperator(symPFunctions::Array, interval::Tuple, t::SymPy.Sym) =
    try
        symL = new(symPFunctions, interval, t)
        check_symLinearDifferentialOperator_input(symL)
        return symL
    catch err
        throw(err)
    end
end

In [17]:
function check_symLinearDifferentialOperator_input(symL::SymLinearDifferentialOperator)
    symPFunctions, interval_tuple, t = symL.symPFunctions, symL.interval, symL.t
    for symPFunc in symPFunctions
        if isa(symPFunc, SymPy.Sym)
            if size(free_symbols(symPFunc)) != (1,) && size(free_symbols(symPFunc)) != (0,)
                throw(StructDefinitionError(:"Only one free symbol is allowed in symP_k"))
            end
        elseif !isa(symPFunc, Number)
            throw(StructDefinitionError(:"symP_k should be SymPy.Sym or Number"))
        end
    end
    for x_i in interval_tuple
        if !isa(x_i, Number)
            throw(StructDefinitionError(:"Interval must consist of numbers"))
        end
    end
    for i=1:length(interval_tuple)-1
        if (interval_tuple[i] < interval_tuple[i+1]) == false
            throw(StructDefinitionError(:"Terms in interval must be strictly increasing"))
        end
    end
    return true
end

check_symLinearDifferentialOperator_input (generic function with 1 method)

**Parameters**
* `symPFunctions`: `Array` of `SymPy.Sym` or `Number`
    * Array $[symP_0, symP_1, \ldots, symP_n]$ of length $n+1$, corresponding to the symbolic linear differential operator $symL$ of order $n$ given by 
    $$symLx = symP_0x^{(n)} + symP_1x^{(n-1)} + \cdots + symP_{n-1}x^{(1)} + symP_n x.$$
* `interval`: `Tuple` 
    * Tuple of $k+1$ numbers $(a=x_0, x_1, \ldots, x_{k-1}, x_k = b)$ corresponding to the real interval $[a=x_0, x_1, \ldots, x_{k-1}, x_k = b]$ on which the symbolic differential operator $symL$ is defined. Note that the length of the tuple is not specified in the above, in order to provide flexibility.
* `t`: `Sym` 
    * Free symbol in each entry of `symPFunctions`.

**Returns**
* `SymLinearDifferentialOperator`
    * Returns a `SymLinearDifferentialOperator` of order $n$ with attributes `symPFunctions`, `interval`, and `t`.

**Example**

In [656]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 0.5, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)

SymLinearDifferentialOperator(Sym[1 t + 1 t^2 + t + 1], (0, 0.5, 1), t)

## `LinearDifferentialOperator(pFunctions, interval, symL)`

A linear differential operator $L$ of order $n$ given by 
$$Lx = p_0x^{(n)} + p_1x^{(n-1)} + \cdots + p_{n-1}x^{(1)} + p_n x$$
is encoded by an $1 \times (n+1)$ array of univariate functions, a tuple $(a= x_0, x_1, \ldots, x_{k-1},x_k = b)$, and its symbolic expression.

In [18]:
struct LinearDifferentialOperator
    pFunctions::Array # Array of julia functions or numbers representing constant functions
    interval::Tuple
    symL::SymLinearDifferentialOperator
    LinearDifferentialOperator(pFunctions::Array, interval::Tuple, symL::SymLinearDifferentialOperator) =
    try
        L = new(pFunctions, interval, symL)
        check_linearDifferentialOperator_input(L)
        return L
    catch err
        throw(err)
    end
end

In [19]:
# Assume symFunc has only one free symbol, as required by the definition of SymLinearDifferentialOperator. 
# That is, assume the input symFunc comes from SymLinearDifferentialOperator.
function check_func_sym_equal(func::Union{Function,Number}, symFunc, interval::Tuple, t::SymPy.Sym) # symFunc should be Union{SymPy.Sym, Number}, but somehow SymPy.Sym gets ignored
    a = interval[1]
    b = interval[length(interval)]
    # Randomly sample 1000 points from (a,b) and check if func and symFunc agree on them
    for i = 1:1000
        # Check endpoints
        if i == 1
            x = a
        elseif i == 2
            x = b
        else
            x = rand(Uniform(a,b), 1)[1,1]
        end
        funcEvalX = evaluate(func, x)
        if isa(symFunc, SymPy.Sym)
            symFuncEvalX = SymPy.N(subs(symFunc,t,x))
            # N() converts SymPy.Sym to Number
            # https://docs.sympy.org/latest/modules/evalf.html
            # subs() works no matter symFunc is Number or SymPy.Sym
        else
            symFuncEvalX = symFunc
        end
        tol = set_tol(funcEvalX, symFuncEvalX)
        if !isapprox(real(funcEvalX), real(symFuncEvalX); atol = real(tol)) ||
            !isapprox(imag(funcEvalX), imag(symFuncEvalX); atol = imag(tol))
            println("x = $x")
            println("symFunc = $symFunc")
            println("funcEvalX = $funcEvalX")
            println("symFuncEvalX = $symFuncEvalX")
            return false
        end
    end
    return true
end

# Check whether the inputs of L are valid.
function check_linearDifferentialOperator_input(L::LinearDifferentialOperator)
    pFunctions, interval, symL = L.pFunctions, L.interval, L.symL
    symPFunctions, t = symL.symPFunctions, symL.t
    # domainC = Complex(a..b, 0..0) # Domain [a,b] represented in the complex plane
    p0 = pFunctions[1]
    # p0Chebyshev = Fun(p0, a..b) # Chebysev polynomial approximation of p0 on [a,b]
    if !check_all(pFunctions, pFunc -> (isa(pFunc, Function) || isa(pFunc, Number)))
        throw(StructDefinitionError(:"p_k should be Function or Number"))
    elseif length(pFunctions) != length(symPFunctions)
        throw(StructDefinitionError(:"Number of p_k and symP_k do not match"))
    elseif interval != symL.interval
        throw(StructDefinitionError(:"Intervals of L and symL do not match"))
    # # Assume p_k are in C^{n-k}. Check whether p0 vanishes on [a,b]. 
    # # roots() in IntervalRootFinding doesn't work if p0 is sth like t*im - 2*im. Neither does find_zero() in Roots.
    # # ApproxFun.roots() 
    # elseif (isa(p0, Function) && (!isempty(roots(p0Chebyshev)) || all(x->x>b, roots(p0Chebyshev)) || all(x->x<b, roots(p0Chebyshev)) || p0(a) == 0 || p0(b) == 0)) || p0 == 0 
    #     throw(StructDefinitionError(:"p0 vanishes on [a,b]"))
    elseif !all(i -> check_func_sym_equal(pFunctions[i], symPFunctions[i], interval, t), 1:length(pFunctions))
        # throw(StructDefinitionError(:"symP_k does not agree with p_k on [a,b]"))
        warn("symP_k does not agree with p_k on the interval") # Make this a warning instead of an error because the functionalities of Julia Functions may be more than those of SymPy objects; we do not want to compromise the functionalities of LinearDifferentialOperator because of the restrictions on SymPy.
    else
        return true
    end
end

check_linearDifferentialOperator_input (generic function with 1 method)

**Parameters**
* `pFunctions`: `Array` of `Function` or `Number` 
    * Array $[p_0, p_1, \ldots, p_n]$ of length $n+1$, corresponding to the linear differential operator $L$ of order $n$ given by
    $$Lx = p_0x^{(n)} + p_1x^{(n-1)} + \cdots + p_{n-1}x^{(1)} + p_n x.$$
* `interval`: `Tuple`
    * Tuple of $k+1$ numbers $(a = x_0, x_1,\ldots, x_{k-1}, x_k = b)$ corresponding to the real interval $[a = x_0, x_1,\ldots, x_{k-1}, x_k = b]$ on which the differential operator $L$ is defined.
* `symL`: `SymLinearDifferentialOperator`
    * Symbolic linear differential operator corresponding to $L$.
    
**Returns**
* `LinearDifferentialOperator`
    * Returns a `LinearDifferentialOperator` of order $n$ with attributes `pFunctions`, `interval`, and `symL`.
    
**Example**

In [359]:
t = symbols("t")
interval = (0, 0.5, 1)

symPFunctions = [1 t+1 t^2+t+1]
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
# Direct construction
pFunctions = [t->1 t->t+1 t->t^2+t+1]
L = LinearDifferentialOperator(pFunctions, interval, symL)

LinearDifferentialOperator(Function[##120#123() ##121#124() ##122#125()], (0, 0.5, 1), SymLinearDifferentialOperator(Sym[1 t + 1 t^2 + t + 1], (0, 0.5, 1), t))

## `VectorMultiBoundaryForm`
A set of homogeneous boundary conditions in vector form
$$Uq = \begin{bmatrix}U_1\\\vdots\\ U_m\end{bmatrix}q = \sum^k_{l=1}\sum_{j=1}^n\begin{bmatrix} M_{1jl}q_l^{(j)}(x_{l-1}) + N_{1jl}q_l^{(j)}(x_{l})\\\vdots\\  M_{mjl}q_l^{(j)}(x_{l-1}) + N_{mjl}q_l^{(j)}(x_{l})\end{bmatrix} = \begin{bmatrix}0\\\vdots\\ 0\end{bmatrix}$$
is encoded by a list of $k+1$ ordered pairs of two linearly independent $m\times n$ matrices $(M_l, N_l)$ where
$$M_l = \begin{bmatrix}M_{11l} & \cdots & M_{1nl}\\ \vdots & \ddots & \vdots\\ M_{m1l} & \cdots & M_{mnl}\end{bmatrix},\quad N_l = \begin{bmatrix}N_{11l} & \cdots & N_{1nl}\\ \vdots & \ddots & \vdots\\ N_{m1l} & \cdots & N_{mnl}\end{bmatrix}.$$

In [20]:
struct VectorMultiBoundaryForm
    MM::Array
    NN::Array
    VectorMultiBoundaryForm(MM::Array, NN::Array) =
    try
        U = new(MM, NN)
        check_vectorMultiBoundaryForm_input(U)
        return U
    catch err
        throw(err)
    end
end

In [21]:
function check_vectorMultiBoundaryForm_input(U::VectorMultiBoundaryForm)
    # M, N = U.M, U.N
    # Avoid Inexact() error when taking rank()
    M = U.MM
    N = U.NN
    checker = Array{Bool}(undef, 1, length(M))
    for i = 1:length(M)
        M_i, N_i = M[i], N[i]
        if !(check_all(M_i, x -> isa(x, Number)) && check_all(N_i, x -> isa(x, Number)))
            throw(StructDefinitionError(:"Entries of M_i, N_i should be Number"))
        elseif size(M_i) != size(N_i)
            throw(StructDefinitionError(:"M_i, N_i dimensions do not match"))
        elseif size(M_i)[1] != size(M_i)[2]
            throw(StructDefinitionError(:"M_i, N_i should be square matrices"))
        elseif LinearAlgebra.rank(hcat(convert(Array{Complex}, M[i]), convert(Array{Complex}, N[i]))) != size(convert(Array{Complex}, M[i]))[1] # rank() throws weird "InexactError()" when taking some complex matrices
            throw(StructDefinitionError(:"Boundary operators not linearly independent"))
        else
            checker[i] = true
        end
    end
    for x in checker
        if !(x == true)
            return false
        end
    return true
    end
end

check_vectorMultiBoundaryForm_input (generic function with 1 method)

**Parameters**
* `M`, `N`: `Array` of `Array`
    * Two arrays whose elements are matrices of the same dimension. These matrices are required to be linearly independent in a sense that $M[i]$ and $N[i]$ are linearly independent for all $i.$

**Returns**
* `VectorMultiBoundaryForm`
    * Returns a `VectorMultiBoundaryForm` with attributes `MM` and `NN`. $MM$ and $NN$ are chosen to avoid confusion with `VectorBoundaryForm` where attributes are `M` and `N`.

**Example**

In [292]:
M1 = [1. 2.; 1. 0.]
M2 = [1. 0.; 0. 2.]
M = [M1, M2]
N1 = [3. 7.; 0. 3.]
N2 = [1. 0.; 0. 10.]
N = [N1, N2]
U = VectorMultiBoundaryForm(M, N)

VectorMultiBoundaryForm(Array{Float64,2}[[1.0 2.0; 1.0 0.0], [1.0 0.0; 0.0 2.0]], Array{Float64,2}[[3.0 7.0; 0.0 3.0], [1.0 0.0; 0.0 10.0]])

# Main functions
Constructs a valid adjoint boundary condition from a given (homogeneous) multi boundary condition based on IMVP-Adjoint-Construction.pdf.

## `get_L(symL)`
Constructs a `LinearDifferentialOperator` from a given `SymLinearDifferentialOperator`.

In [22]:
function get_L(symL::SymLinearDifferentialOperator)
    symPFunctions, interval, t = symL.symPFunctions, symL.interval, symL.t
    if check_all(symPFunctions, x->!isa(x, SymPy.Sym))
        pFunctions = symPFunctions
    else
        pFunctions = sym_to_func.(symPFunctions)
    end
    L = LinearDifferentialOperator(pFunctions, interval, symL)
    return L
end

get_L (generic function with 1 method)

**Parameters**
* `symL`: `SymLinearDifferentialOperator`
    * Symbolic linear differential operator to be converted.

**Returns**
* `get_L`: `LinearDifferentialOperator`
    * Returns the linear differential operator converted from `symL`.

**Example**

In [360]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 0.5, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
L = get_L(symL)

LinearDifferentialOperator(getfield(Main, Symbol("#func#85")){Sym,Array{Sym,1}}[#func#85{Sym,Array{Sym,1}}(1, Sym[]) #func#85{Sym,Array{Sym,1}}(t + 1, Sym[t]) #func#85{Sym,Array{Sym,1}}(t^2 + t + 1, Sym[t])], (0, 0.5, 1), SymLinearDifferentialOperator(Sym[1 t + 1 t^2 + t + 1], (0, 0.5, 1), t))

## `get_URank(U)`
Computes the rank of a vector boundary form $U$ by computing the equivalent $\text{rank}(M:N)$, where $M, N$ are the matrices associated with $U$ and
$$(M:N) = \begin{bmatrix}M_{11} & \cdots & M_{1n} & N_{11} & \cdots & N_{1n}\\ \vdots & \ddots & \vdots & \vdots & \ddots & \vdots\\ M_{m1} & \cdots & M_{mn} & N_{m1} & \cdots & N_{mn}\end{bmatrix}.$$

In [23]:
function get_URank(U::VectorMultiBoundaryForm)
    # Avoid InexactError() when taking hcat() and rank()
    M = U.MM
    N = U.NN
    checker = Array{Int64}(undef, 1, length(M))
    for i=1:length(M) 
        M_i = convert(Array{Complex}, M[i])
        N_i = convert(Array{Complex}, N[i])
        MHcatN = hcat(M_i, N_i)
        checker[i] = LinearAlgebra.rank(MHcatN)
    end
    if all(y->y == checker[1], checker) == true
        return checker[1]
    else
        throw(StructDefinitionError(:"Matrices M_i, N_i are not appropriate."))
    end
end

get_URank (generic function with 1 method)

**Parameters**
* `U`: `VectoBoundaryForm`
    * Vector boundary form whose rank is to be computed.

**Returns**
* `get_URank`: `Number`
    * Returns the rank of `U`.

**Example**

In [293]:
M1 = [1. 2.; 1. 0.]
M2 = [1. 0.; 0. 2.]
M = [M1, M2]
N1 = [3. 7.; 0. 3.]
N2 = [1. 0.; 0. 10.]
N = [N1, N2]
U = VectorMultiBoundaryForm(M, N)
get_URank(U)

2

## `get_Uc(U)`
Given a vector boundary form $U = \begin{bmatrix}U_1\\ \vdots\\ U_m\end{bmatrix}$ of rank $m$, finds a complementary form $U_c = \begin{bmatrix}U_{m+1}\\ \vdots\\ U_{2n}\end{bmatrix}$ of rank $2n-m$ such that $\begin{bmatrix}U_1\\ \vdots\\ U_{2n}\end{bmatrix}$ has rank $2n$.

In [24]:
# Find Uc, a complementary form of U
function get_Uc(U::VectorMultiBoundaryForm)
    try
        check_vectorMultiBoundaryForm_input(U)
        M, N = copy(U.MM), copy(U.NN)
        P = convert.(Array{Complex}, M)
        Q = convert.(Array{Complex}, N)       
        n = get_URank(U)
        I = complex(Matrix{Float64}(LinearAlgebra.I, 2n, 2n))
        for i=1:length(M) 
            M_i = convert(Array{Complex}, M[i])
            N_i = convert(Array{Complex}, N[i])
            mat_i = hcat(M_i, N_i)
            for k = 1:(2*n)
                newMat_k = vcat(mat_i, I[k:k,:])
                if LinearAlgebra.rank(newMat_k) == LinearAlgebra.rank(mat_i) + 1
                mat_i = newMat_k
            end
        end
        UcHcat_i = mat_i[(n+1):(2n),:]
        P[i] = UcHcat_i[:,1:n]
        Q[i] = UcHcat_i[:,(n+1):(2n)]
        end
        Uc = VectorMultiBoundaryForm(P, Q)
        return Uc
    catch err
        return err
    end
end

get_Uc (generic function with 1 method)

**Parameters**
* `U`: `VectoBoundaryForm`
    * Vector boundary form whose complementary boundary form is to be found.

**Returns**
* `get_Uc`: `VectorBoundaryForm`
    * Returns a vectory boundary form complementary to `U`.

**Example**

In [313]:
M1 = [1. 2.; 1. 0.]
M2 = [1. 0.; 0. 2.]
M = [M1, M2]
N1 = [3. 7.; 0. 3.]
N2 = [1. 0.; 0. 10.]
N = [N1, N2]
U = VectorMultiBoundaryForm(M, N)
Uc = get_Uc(U)
println(Uc)

VectorMultiBoundaryForm(Array{Complex,2}[[1.0+0.0im 0.0+0.0im; 0.0+0.0im 1.0+0.0im], [1.0+0.0im 0.0+0.0im; 0.0+0.0im 1.0+0.0im]], Array{Complex,2}[[0.0+0.0im 0.0+0.0im; 0.0+0.0im 0.0+0.0im], [0.0+0.0im 0.0+0.0im; 0.0+0.0im 0.0+0.0im]])


## `get_H(U, Uc)`
Given a vector boundary form $U$ and a complementary vector boundary form $U_c$, constructs 
$$H = \begin{bmatrix}M&N\\ M_c & N_c\end{bmatrix},$$
where $M, N$ are the matrices associated with $U$ and $M_c, N_c$ are associated with $U_c$.

In [25]:
function get_H(U::VectorMultiBoundaryForm, Uc::VectorMultiBoundaryForm)
    M, N = copy(U.MM), copy(U.NN) # need to use copies of U and Uc, not the actual things
    P, Q = copy(Uc.MM), copy(Uc.NN)
    H = convert.(Array{Complex}, M)
    for i=1:length(M)
        MHcatN = hcat(convert(Array{Complex}, M[i]), convert(Array{Complex}, N[i]))
        McHcatNc = hcat(convert(Array{Complex}, P[i]), convert(Array{Complex}, Q[i]))
        H[i] = vcat(MHcatN, McHcatNc)
    end
    return H
end

get_H (generic function with 1 method)

**Parameters**
* `U`: `VectorBoundaryForm`
    * Vector boundary form.
* `Uc`: `VectorBoundaryForm`
    * Vector boundary form complementary to `U`.

**Returns**
* `get_H`: `Array`
    * Returns the matrix $H$ defined above.

**Example**

In [314]:
M1 = [1. 2.; 1. 0.]
M2 = [1. 0.; 0. 2.]
M = [M1, M2]
N1 = [3. 7.; 0. 3.]
N2 = [1. 0.; 0. 10.]
N = [N1, N2]
U = VectorMultiBoundaryForm(M, N)
Uc = get_Uc(U)
get_H(U, Uc)

2-element Array{Array{Complex,2},1}:
 [1.0+0.0im 2.0+0.0im 3.0+0.0im 7.0+0.0im; 1.0+0.0im 0.0+0.0im 0.0+0.0im 3.0+0.0im; 1.0+0.0im 0.0+0.0im 0.0+0.0im 0.0+0.0im; 0.0+0.0im 1.0+0.0im 0.0+0.0im 0.0+0.0im] 
 [1.0+0.0im 0.0+0.0im 1.0+0.0im 0.0+0.0im; 0.0+0.0im 2.0+0.0im 0.0+0.0im 10.0+0.0im; 1.0+0.0im 0.0+0.0im 0.0+0.0im 0.0+0.0im; 0.0+0.0im 1.0+0.0im 0.0+0.0im 0.0+0.0im]

## `get_symPDerivMatrix(L; symbolic)`
Given a `LinearDifferentialOperator` `L` where `L.pFunctions` is the array
$$[p_0, p_1, \ldots, p_n],$$
constructs an $n\times n$ matrix whose $(i+1)(j+1)$-entry is a function corresponding to the $j$th derivative of $p_i$:
$$\begin{bmatrix}p_0 & \cdots & p_0^{(n-1)}\\ \vdots & \ddots & \vdots\\ p_{n-1} & \cdots & p_{n-1}^{(n-1)}\end{bmatrix}.$$

In [26]:
function get_pDerivMatrix(L::LinearDifferentialOperator; symbolic = false)
    if symbolic
        symL = L.symL
        symPFunctions, t = symL.symPFunctions, symL.t
        n = length(symPFunctions)-1
        symPDerivMatrix = Array{SymPy.Sym}(undef, n,n)
        pFunctionSymbols = symPFunctions
        for i in 0:(n-1)
            for j in 0:(n-1)
                index, degree = i, j
                symP = pFunctionSymbols[index+1]
                # If symP is not a Sympy.Sym object (e.g., is a Number instead), then cannot use get_deriv()
                if !isa(symP, SymPy.Sym)
                    if degree > 0
                        symPDeriv = 0
                    else
                        symPDeriv = symP
                    end
                else
                    symPDeriv = get_deriv(symP, degree)
                end
                symPDerivMatrix[i+1,j+1] = symPDeriv
            end
        end
        return symPDerivMatrix
    else
        symPDerivMatrix = get_pDerivMatrix(L; symbolic = true)
        n = length(L.pFunctions)-1
        pDerivMatrix = sym_to_func.(symPDerivMatrix)
    end
    return pDerivMatrix
end

get_pDerivMatrix (generic function with 1 method)

**Parameters**
* `L`: `LinearDifferentialOperator`
    * Linear differential operator whose `pDerivMatrix` is to be constructed.
* `symbolic*`: `Bool`
    * Boolean indicating whether the output is symbolic.

**Returns**
* `get_pDerivMatrix`: `Array` of `Function`, `Number` or `SymPy.Sym`
    * Returns an $n\times n$ matrix whose $(i+1)(j+1)$-entry is
        * the $j$th derivative of $p_i$ (`L.pFunctions[i]`) if `symbolic = false`, or
        * the symbolic expression of the $j$th derivative of $p_i$ (`L.symL.symPFunctions[i]`) if `symbolic = true`.

**Example**

In [361]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 0.5, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
L = get_L(symL)

tVal = 2
pDerivMatrix = get_pDerivMatrix(L; symbolic = false)
println("pDerivMatrix($tVal) = $(evaluate.(pDerivMatrix,tVal))")

symPDerivMatrix = get_pDerivMatrix(L; symbolic = true)

pDerivMatrix(2) = [1 0; 3 1]


2×2 Array{Sym,2}:
     1  0
 t + 1  1

## `get_Bjk(L, j, k; symbolic, pDerivMatrix)`
Given a `LinearDifferentialOperator` `L` of order $n$, for $j, k \in \{1,\ldots,n\}$, computes $B_{jk}$ defined as
$$B_{jk}(t) := \sum_{\ell=j-1}^{n-k}\binom{\ell}{j-1}p^{(\ell-j+1)}_{n-k-\ell}(t)(-1)^\ell.$$

In [27]:
# Find Bjk using explicit formula
function get_Bjk(L::LinearDifferentialOperator, j::Int, k::Int; symbolic = false, pDerivMatrix = get_pDerivMatrix(L; symbolic = symbolic))
    n = length(L.pFunctions)-1
    if j <= 0 || j > n || k <= 0 || k > n
        throw("j, k should be in {1, ..., n}")
    end
    sum = 0
    if symbolic
        symPDerivMatrix = get_pDerivMatrix(L; symbolic = true)
        for l = (j-1):(n-k)
            summand = binomial(l, j-1) * symPDerivMatrix[n-k-l+1, l-j+1+1] * (-1)^l
            sum += summand
        end
    else
        for l = (j-1):(n-k)
            summand = mult_func(binomial(l, j-1) * (-1)^l, pDerivMatrix[n-k-l+1, l-j+1+1])
            sum = add_func(sum, summand)
        end
    end
    return sum
end

get_Bjk (generic function with 1 method)

**Parameters**
* `L`: `LinearDifferentialOperator`
    * Linear differential operator whose `L.pFunctions` are to become the $p_{n-k-l}^{l-j+1}$ in $B_{jk}(t)$.
* `j`, `k`: `Int`
    * Integers corresponding to the $j$ and $k$ in $B_{jk}$.
* `symbolic*`: `Bool`
    * Boolean indicating whether the output is symbolic.
* `pDerivMatrix*`: `Array`
    * If `symbolic = false`, an $n\times n$ matrix whose $(i+1)(j+1)$-entry is the $j$th derivative of $p_i$ (`L.pFunctions[i]`) implemented as a `Function`, `Number`, or `SymPy.Sym`. Default to the output of `get_pDerivMatrix(L)`.

**Returns**
* `get_Bjk`: `SymPy.Sym`, `Function`, or `Number`
    * Returns $B_{jk}(t)$ defined above, 
        * as `Function` if `symbolic = false`, or
        * as `SymPy.Sym` object if `symbolic = true`, where each $p_i$ is the generic expression $p_i(t)$.
        
**Example**

In [362]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 0.5, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
L = get_L(symL)

j, k = 1, 1
BjkSym = get_Bjk(L, j, k; symbolic = true)

t + 1

In [363]:
tVal = 1
Bjk = get_Bjk(L, j, k; symbolic = false)
println("Bjk($tVal) = $(Bjk(tVal))")
println("BjkSym($tVal) = $(evaluate.(BjkSym, tVal))")

Bjk(1) = 2
BjkSym(1) = 2


## `get_B(L; symbolic, pDerivMatrix)`

Given a `LinearDifferentialOperator` `L` where `L.pFunctions` is the array
$$[p_0, p_1, \ldots, p_n],$$ 
constructs the matrix $B(t)$ whose $ij$-entry is given by
$$B_{jk}(t) := \sum_{\ell=j-1}^{n-k}\binom{\ell}{j-1}p^{(\ell-j+1)}_{n-k-\ell}(t)(-1)^\ell.$$

In [28]:
function get_B(L::LinearDifferentialOperator; symbolic = false, pDerivMatrix = get_pDerivMatrix(L; symbolic = symbolic))
    n = length(L.pFunctions)-1
    B = Array{Union{Function, Number, SymPy.Sym}}(undef, n, n)
    for j = 1:n
        for k = 1:n
            B[j,k] = get_Bjk(L, j, k; symbolic = symbolic, pDerivMatrix = pDerivMatrix)
        end
    end
    return B
end

get_B (generic function with 1 method)

**Parameters**
* `L`: `LinearDifferentialOperator`
    * Linear differential operator whose `L.pFunctions` are to become the $p_{n-k-l}^{l-j+1}$ in $B_{jk}(t)$.
* `symbolic*`: `Bool`
    * Boolean indicating whether the output is symbolic.
* `substitute*`: `Bool`
    * If `symbolic = true`, boolean indicating whether to substitute the symbolic expression of $p_i$ in `L.pFunctions` for the generic expression $p_i(t)$ created using `SymFunction("pi")(t)`. If `symbolic = false`, the value of `substitute` does not matter.
* `pDerivMatrix*`: `Array`
    * If `symbolic = false`, the non-symbolic version of `symPDerivMatrix`, i.e., an $n\times n$ matrix whose $(i+1)(j+1)$-entry is the $j$th derivative of $p_i$ (`L.pFunctions[i]`) implemented as a `Function` or `Number`.

**Returns**
* `get_B`: `Array` of `Function`, `SymPy.Sym`, or `Number`
    * Returns $B(t)$ defined above, where $B_{jk}(t)$ is
        * `Function` if `symbolic = false`, or
        * `SymPy.Sym` object if `symbolic = true`, where each $p_i$ is
            * the generic expression $p_i(t)$ if `substitute = false`, or
            * the symbolic expression of $p_i(t)$ (`L.symL.symPFunctions[i]`) if `substitute = true`.

**Example**

In [364]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 0.5, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
L = get_L(symL)

BSym = get_B(L; symbolic = true)
prettyPrint.(BSym)

2×2 Array{Sym,2}:
 t + 1  1
    -1  0

In [365]:
B = get_B(L; symbolic = false)
tVal = 1
println("B($tVal) = $(evaluate.(B, tVal))")
println("BSym($tVal) = $(evaluate.(BSym, tVal))")

B(1) = [2 1; -1 0]
BSym(1) = Number[2 1; -1 0]


## `get_BHat(L, B)`
Given a `LinearDifferentialOperator` `L` where `L.pFunctions` is the array
$$[p_0, p_1, \ldots, p_n]$$
and `L.interval` is $[x_0, x_1, x_2, \ldots, x_{k-1}, x_k]$, constructs a tuple of 
$$\{ \widehat{B_l}\}_{l=1}^k, \mbox{ where } \widehat{B_l} :=\begin{bmatrix}-B(x_{l-1}) & 0_n\\0_n & B(x_l)\end{bmatrix}.$$

In [29]:
# Construct B_hat. Since all entries of B_hat are evaluated, BHat is a numeric matrix.
function get_BHat(L::LinearDifferentialOperator, B::Array)
    pFunctions, interval = L.pFunctions, L.interval
    k = length(interval) - 1
    BHats_Array = Array{Any}(undef, 1, k)
    n = length(pFunctions)-1
    for i=1:k
        BHat = Array{Float64, 2}(undef, 2n, 2n)
        BHat = convert(Array{Complex}, BHat)
        BEvalA = evaluate.(B, interval[i]) # B at x_{l-1} 
        BEvalB = evaluate.(B, interval[i+1]) # B at x_l
        BHat[1:n,1:n] = -BEvalA
        BHat[(n+1):(2n),(n+1):(2n)] = BEvalB
        BHat[1:n, (n+1):(2n)] = zeros(n, n)
        BHat[(n+1):(2n), 1:n] = zeros(n, n)
        BHats_Array[i] = BHat
    end
    return BHats_Array
end

get_BHat (generic function with 1 method)

**Parameters**
* `L`: `LinearDifferentialOperator`
    * Linear differential operator whose `L.pFunctions` are to become the $p_{n-k-l}^{l-j+1}$ in $B_{jk}(t)$.
* `B`: `Array` of `Number`
    * Output of `get_B(L; symbolic = false)`.
    
**Returns**
* `get_BHat`: `Array` of `Number`
    * Returns $\hat{B}$ defined above.

**Example**

In [366]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 0.5, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
L = get_L(symL)

B = get_B(L; symbolic = false)
BHat_Arr = get_BHat(L, B)

1×2 Array{Any,2}:
 Complex[-1+0im -1+0im 0.0+0.0im 0.0+0.0im; 1+0im 0+0im 0.0+0.0im 0.0+0.0im; 0.0+0.0im 0.0+0.0im 1.5+0.0im 1+0im; 0.0+0.0im 0.0+0.0im -1+0im 0+0im]  …  Complex[-1.5+0.0im -1+0im 0.0+0.0im 0.0+0.0im; 1+0im 0+0im 0.0+0.0im 0.0+0.0im; 0.0+0.0im 0.0+0.0im 2+0im 1+0im; 0.0+0.0im 0.0+0.0im -1+0im 0+0im]

## `get_J(BHat_Array, H_Array)`
Given $\{\widehat{B_l}\}_{l=1}^k$ and $\{H_l\}_{l=1}^k$, constructs $\{ J_l \}_{l=1}$ defined as 
$$J_l:=(\widehat{B_l} H_l^{-1})^\star$$
where $^*$ denotes conjugate transpose.

In [30]:
function get_J(BHat_Array, H_Array)
    k = length(BHat_Array)
    J = Array{Any}(undef, 1, k)
    for i=1:k
        BHat_i = BHat_Array[i]
        H_i = H_Array[i] 
        n = size(H_i)[1]
        H_i = convert(Array{Complex}, H_i)
        J_i = (BHat_i * inv(H_i))'
        J[i] = J_i
    end
    return J
end

get_J (generic function with 1 method)

**Parameters**
* `BHat_Array`: `Array` of `Array`
    * Output of `get_BHat()`.
* `H_Array`: `Array` of `Array`
    * Output of `get_H()`.

**Returns**
* `get_J`: `Array` of `Array`
    * Returns a tuple of $J_l$ defined above.

**Example**

In [367]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 0.5, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
L = get_L(symL)

B = get_B(L; symbolic = false)
BHat_Arr = get_BHat(L, B)

M1 = [1. 2.; 1. 0.]
M2 = [1. 0.; 0. 2.]
M = [M1, M2]
N1 = [3. 7.; 0. 3.]
N2 = [1. 0.; 0. 10.]
N = [N1, N2]
U = VectorMultiBoundaryForm(M, N)
Uc = get_Uc(U)
H_Arr = get_H(U, Uc)
J_Arr = get_J(BHat_Arr, H_Arr)

1×2 Array{Any,2}:
 Any[0.0-0.0im 0.0-0.0im 0.5-0.0im -0.333333-0.0im; 0.0-0.0im 0.0-0.0im -0.833333-0.0im 0.777778-0.0im; -1.0-0.0im 1.0-0.0im 0.333333-0.0im -0.444444-0.0im; -1.0-0.0im 0.0-0.0im -1.0-0.0im 0.666667-0.0im]  …  Any[0.0-0.0im 0.0-0.0im 2.0-0.0im -1.0-0.0im; 0.0-0.0im 0.0-0.0im 0.1-0.0im 0.0-0.0im; -1.5-0.0im 1.0-0.0im -2.0-0.0im 1.0-0.0im; -1.0-0.0im 0.0-0.0im -0.2-0.0im 0.0-0.0im]

## `get_adjoint_Candidate(J_Array)`
Given $\{ J_l \}_{l=1}^k$, constructs a candidate adjoint vector boundary form $U^+$ from two tuples $\{P^\star_l\}_{l=1}^k$, $\{Q^\star_l\}_{l=1}^k,$ whose elements are matrices $P^\star_l$, $Q^\star_l$, which are the lower-left $n\times n$ submatrix of $J_l$, and the lower-right $n\times n$ submatrix of $J_l$, respectively.

In [31]:
# Construct U+
function get_adjoint_Candidate(J_Array)
    PStar = Array{Any}(undef, 1, length(J_Array))
    QStar = Array{Any}(undef, 1, length(J_Array))
    for i=1:length(J_Array)
        J_i = J_Array[i]
        n = convert(Int, size(J_i)[1]/2)
        J = convert(Array{Complex}, J_i)
        PStar[i] = J_i[(n+1):2n,1:n]
        QStar[i] = J_i[(n+1):2n, (n+1):2n]
    end
    adjointU = VectorMultiBoundaryForm(PStar, QStar)
    return adjointU
end

get_adjoint_Candidate (generic function with 1 method)

**Parameters**
* `J`: `Array` of `Array`
    * Output of `get_J`.

**Returns**
* `get_adjoint`: `VectorMultiBoundaryForm`
    * Returns $U^+$ defined above.

**Example**

In [374]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 0.5, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
L = get_L(symL)

B = get_B(L; symbolic = false)
BHat_Arr = get_BHat(L, B)

M1 = [1. 2.; 1. 0.]
M2 = [1. 0.; 0. 2.]
M = [M1, M2]
N1 = [3. 7.; 0. 3.]
N2 = [1. 0.; 0. 10.]
N = [N1, N2]
U = VectorMultiBoundaryForm(M, N)
Uc = get_Uc(U)
H_Arr = get_H(U, Uc)
J_Arr = get_J(BHat_Arr, H_Arr)
adjoint = get_adjoint_Candidate(J_Arr)

VectorMultiBoundaryForm(Any[Any[-1.0-0.0im 1.0-0.0im; -1.0-0.0im 0.0-0.0im] Any[-1.5-0.0im 1.0-0.0im; -1.0-0.0im 0.0-0.0im]], Any[Any[0.333333-0.0im -0.444444-0.0im; -1.0-0.0im 0.666667-0.0im] Any[-2.0-0.0im 1.0-0.0im; -0.2-0.0im 0.0-0.0im]])

## `get_symXi(L; symbolic, xSym)`
Given a `LinearDifferentialOperator` `L` of order $n$ in the differential equation $Lx=0$, constructs $\xi(t)$, which is defined as the vector of derivatives of $x(t)$
$$\xi(t) := \begin{bmatrix}x(t)\\ x^{(1)}(t)\\ x^{(2)}(t)\\ \vdots\\ x^{(n-1)}(t)\end{bmatrix}.$$

In [32]:
function get_xi(L::LinearDifferentialOperator; symbolic = true, xSym= nothing)
    if symbolic
        t = L.symL.t
        n = length(L.pFunctions)-1
        symXi = Array{SymPy.Sym}(undef, n,1)
        if isa(xSym, Nothing)
            throw(error("xSymrequired"))
        else
            for i = 1:n
                symXi[i] = get_deriv(xSym, i-1)
            end
            return symXi
        end
    else
        if isa(xSym, Nothing)
            throw(error("xSym required"))
        elseif !isa(xSym, SymPy.Sym) && !isa(xSym, Number)
            throw(error("xSym should be SymPy.Sym or Number"))
        else
            symXi = get_xi(L; symbolic = true, xSym = xSym)
            xi = sym_to_func.(symXi)
        end
    end
end

get_xi (generic function with 1 method)

**Parameters**
* `L`: `LinearDifferentialOperator`
    * Linear differential operator in the differential equation $Lx=0$; derivatives of $x(t)$ will be entries of $\xi(t)$.
* `symbolic`: `Bool`
    * Boolean indicating whether the output is symbolic.
* `substitute*`: `Bool`
    * If `symbolic = true`, boolean indicating whether to substitute the symbolic expression of $x(t)$ for the generic expression created using `SymFunction`.
* `xSym*`: `SymPy.Sym`
    * If `substitute = true`, symbolic expression of $x(t)$ to replace the generic expression with.

**Returns**
* `get_xi`: `Array` of `SymPy.Sym`
    * Returns an array whose $i$th entry is
        * the generic expression $\displaystyle\frac{d^{i-1}}{dt^{i-1}}x(t)$ if `substitute = false`, or
        * the symbolic expression of the ($i-1$)th derivative of $x(t)$ if `substitute = true`.

**Example**

In [376]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 0.5, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
L = get_L(symL)

LinearDifferentialOperator(getfield(Main, Symbol("#func#85")){Sym,Array{Sym,1}}[#func#85{Sym,Array{Sym,1}}(1, Sym[]) #func#85{Sym,Array{Sym,1}}(t + 1, Sym[t]) #func#85{Sym,Array{Sym,1}}(t^2 + t + 1, Sym[t])], (0, 0.5, 1), SymLinearDifferentialOperator(Sym[1 t + 1 t^2 + t + 1], (0, 0.5, 1), t))

In [377]:
xSym = t^2+2t
symXi = get_xi(L; symbolic = true, xSym = xSym)

2×1 Array{Sym,2}:
 t^2 + 2*t
   2*t + 2

In [378]:
xi = get_xi(L; symbolic=false, xSym = xSym)
tVal = 1
println("xi($tVal) = $(evaluate.(xi, tVal))")
println("symXi($tVal) = $(evaluate.(symXi, tVal))")

xi(1) = [3; 4]
symXi(1) = [3; 4]


## `get_Ux(L, U, xSym)`
Given a `LinearDifferentialOperator` `L` and a `VectorBoundaryForm` `U`, constructs the left hand side of the homogeneous boundary condition $Uq=0$, where
$$\begin{align*}
    Uq &= \sum_{l=1}^k \begin{bmatrix}
        \sum_{j=1}^n (M_{1jl}q^{(j-1)}(x_{l-1}) + N_{1jl}q^{(j-1)}(x_{l}))\\
        \vdots\\
        \sum_{j=1}^n (M_{mjl}q^{(j-1)}(x_{l-1}) + N_{mjl}q^{(j-1)}(x_{l}))
    \end{bmatrix}\\
    &= \sum_{l=1}^k \begin{bmatrix}
        M_{11l} & \cdots & M_{1nl} & N_{11} & \cdots & N_{1nl}\\
        \vdots &  & \vdots & \vdots & & \vdots\\
        M_{m1l} & \cdots & M_{mnl} & N_{m1} & \cdots & N_{mnl}
    \end{bmatrix} \begin{bmatrix}q(x_{l-1})\\\vdots\\q^{(n-1)}(x_{l-1})\\ q(x_l)\\\vdots\\q^{(n-1)}(x_l)\end{bmatrix}\\
    &= \sum_{l=1}^k (M_l:N_l)\begin{bmatrix}
        \xi(x_{l-1})\\
        \xi(x_l)
    \end{bmatrix}.
\end{align*}$$

In [33]:
function get_Ux(L::LinearDifferentialOperator, U::VectorMultiBoundaryForm, xSym)
    interval = L.interval
    k = length(interval)-1
    xi = get_xi(L; symbolic = false, xSym = xSym)
    summand = zeros(k,1)
    M, N = copy(U.MM), copy(U.NN)
    for i=1:k
        xiEvalA = evaluate.(xi, interval[i])
        xiEvalB = evaluate.(xi, interval[i+1])
        Ux = M[i]*xiEvalA + N[i]*xiEvalB
        summand = Ux + summand 
    end
    return summand
end

get_Ux (generic function with 1 method)

In [380]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 0.5, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
L = get_L(symL)

M1 = [1. 2.; 1. 0.]
M2 = [1. 0.; 0. 2.]
M = [M1, M2]
N1 = [3. 7.; 0. 3.]
N2 = [1. 0.; 0. 10.]
N = [N1, N2]
U = VectorMultiBoundaryForm(M, N)

VectorMultiBoundaryForm(Array{Float64,2}[[1.0 2.0; 1.0 0.0], [1.0 0.0; 0.0 2.0]], Array{Float64,2}[[3.0 7.0; 0.0 3.0], [1.0 0.0; 0.0 10.0]])

In [381]:
xSym = t^2+2t
Ux = get_Ux(L, U, xSym)
println("U($xSym) = $Ux")

U(t^2 + 2*t) = [33.0; 55.0]


## `check_adjoint(L, U, adjointU, B)`
Given a multi-point boundary value problem
$$Lq = 0,\quad Uq=0$$
with linear differential operator $L$ and vector multi-boundary form $U$, a candidate adjoint vector multi-boundary form $U^+$, and the matrix $B$ associated with $L$, checks whether the boundary condition
$$U^+q = 0$$
is indeed adjoint to the boundary condition
$$Uq=0.$$

In [34]:
function check_adjoint(L::LinearDifferentialOperator, U::VectorMultiBoundaryForm, adjointU::VectorMultiBoundaryForm, B::Array)
    interval = L.interval
    k = length(interval)-1
    M, N = U.MM, U.NN
    P, Q = (adjointU.MM)', (adjointU.NN)'
    # Avoid InexactError() when taking inv()
    checker = Array{Bool}(undef, 1, k)
    for i=1:k
        BEvalA = convert(Array{Complex}, evaluate.(B, interval[i]))
        BEvalB = convert(Array{Complex}, evaluate.(B, interval[i+1]))
        left = M[i] * inv(BEvalA) * P[i]
        right = N[i] * inv(BEvalB) * Q[i]
        tol = set_tol(left, right)
        checker[i] = all(j -> isapprox(left[j], right[j]; atol = tol), 1:length(left)) # Can't use == to deterimine equality because left and right are arrays of floats
    end
    for x in checker
        if x != true
            return false
        end
        return true
    end
end

check_adjoint (generic function with 1 method)

**Parameters**
* `L`: `LinearDifferentialOperator`
    * Linear differential operator in the differential equation $Lx=0$.
* `U`: `VectorBoundaryForm`
    * Vector boundary form in the boundary condition $Ux=0$.
* `adjointU`: `VectorBoundaryForm`
    * Vector boundary form in the candidate adjoint boundary condition $U^+x=0$.
* `B`: `Array` of `Number`
    * Output of `get_B(L)`.

**Returns**
* `check_adjoint`: `Bool`
    * Returns 
        * `true` if `adjointU` is indeed adjoint to `U`, or
        * `false` otherwise.

**Example**

In [385]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 0.5, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
L = get_L(symL)

B = get_B(L; symbolic = false)
BHat_Arr = get_BHat(L, B)

M1 = [1. 2.; 1. 0.]
M2 = [1. 0.; 0. 2.]
M = [M1, M2]
N1 = [3. 7.; 0. 3.]
N2 = [1. 0.; 0. 10.]
N = [N1, N2]
U = VectorMultiBoundaryForm(M, N)
Uc = get_Uc(U)

H_Arr = get_H(U, Uc)
J_Arr = get_J(BHat_Arr, H_Arr)
adjointU = get_adjoint_Candidate(J_Arr)

check_adjoint(L, U, adjointU, B)

true

## `get_adjointU(L, U, pDerivMatrix)`

Given a multi-point boundary value problem
$$Lq = p_0q^{(n)} + p_1q^{(n-1)} + \cdots + p_{n-1}q^{(1)} + p_n q = 0,\quad Uq=0$$
with linear differential operator $L$ and vector boundary form $U$, an $n\times n$ matrix of derivatives
$$\begin{bmatrix}p_0 & \cdots & p_0^{(n-1)}\\ \vdots & \ddots & \vdots\\ p_{n-1} & \cdots & p_{n-1}^{(n-1)}\end{bmatrix},$$
construct $U^+$ such that the boundary condition $U^+q=0$ is adjoint to the original boundary condition $Uq=0$.

In [35]:
function get_adjointU(L::LinearDifferentialOperator, U::VectorMultiBoundaryForm, pDerivMatrix=get_pDerivMatrix(L))
    B = get_B(L; pDerivMatrix = pDerivMatrix)
    BHat_Arr = get_BHat(L, B)
    Uc = get_Uc(U)
    H_Arr = get_H(U, Uc)
    J_Arr = get_J(BHat_Arr, H_Arr)
    adjointU = get_adjoint_Candidate(J_Arr)
    if check_adjoint(L, U, adjointU, B)
        return adjointU
    else
        throw(error("Adjoint found not valid"))
    end
end

get_adjointU (generic function with 2 methods)

**Parameters**
* `L`: `LinearDifferentialOperator`
    * Linear differential operator in the differential equation $Lq=0$.
* `U`: `VectorMultiBoundaryForm`
    * Vector multi boundary form in the boundary condition $Uq=0$.
* `pDerivMatrix`: `Array` of `Function`, `Number`, or `SymPy.#`
    * An $n\times n$ matrix defined above, which can be
        * output of `get_pDerivMatrix` (`SymPy.#`), or
        * user input.

**Returns**
* `get_adjointU`: `VectorMultiBoundaryForm`
    * Returns a valid vector boundary form $U^+$ such that the boundary condition $U^+q=0$ is adjoint to $Uq=0$.

**Example**

In [389]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 0.5, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
# pFunctions = [t->1 t->t+1 t->t^2+t+1]
# L = LinearDifferentialOperator(pFunctions, interval, symL)
L = get_L(symL)
M1 = [1. 2.; 1. 0.]
M2 = [1. 0.; 0. 2.]
M = [M1, M2]
N1 = [3. 7.; 0. 3.]
N2 = [1. 0.; 0. 10.]
N = [N1, N2]
U = VectorMultiBoundaryForm(M, N)

adjointU = get_adjointU(L, U)

pDerivMatrix = get_pDerivMatrix(L)

2×2 Array{getfield(Main, Symbol("#func#85")){Sym,Array{Sym,1}},2}:
 #func#85{Sym,Array{Sym,1}}(1, Sym[])       …  #func#85{Sym,Array{Sym,1}}(0, Sym[])
 #func#85{Sym,Array{Sym,1}}(t + 1, Sym[t])     #func#85{Sym,Array{Sym,1}}(1, Sym[])

In [390]:
get_adjointU(L, U, pDerivMatrix)

VectorMultiBoundaryForm(Any[Any[-1.0-0.0im 1.0-0.0im; -1.0-0.0im 0.0-0.0im] Any[-1.5-0.0im 1.0-0.0im; -1.0-0.0im 0.0-0.0im]], Any[Any[0.333333-0.0im -0.444444-0.0im; -1.0-0.0im 0.666667-0.0im] Any[-2.0-0.0im 1.0-0.0im; -0.2-0.0im 0.0-0.0im]])

# Helper Functions for Testing

## `generate_symPFunctions`

In [36]:
function generate_symPFunctions(n; random = false, constant = false)
    global t = symbols("t")
    if random
        symPFunctions = Array{Number}(undef, 1,n+1)
        for i = 1:(n+1)
            seed = rand(0:1)
            if seed == 0 # constant
                symPFunctionRe = rand(Uniform(1.0,10.0), 1, 1)[1]
                symPFunctionIm = rand(Uniform(1.0,10.0), 1, 1)[1]
                symPFunction = symPFunctionRe + symPFunctionIm*im
            else # variable
                coeffsNo = rand(1:5)
                pFunctionCoeffsRe = rand(Uniform(1.0,10.0), 1, coeffsNo)
                pFunctionCoeffsIm = rand(Uniform(1.0,10.0), 1,  coeffsNo)
                pFunctionCoeffs = pFunctionCoeffsRe + pFunctionCoeffsIm*im
                symPFunction = sum([pFunctionCoeffs[i+1]*t^(length(pFunctionCoeffs)-1-i) for i in 0:(length(pFunctionCoeffs)-1)])
            end
            symPFunctions[i] = symPFunction
        end
    else
        if constant # constant
            symPFunctionsRe = rand(Uniform(1.0,10.0), 1, (n+1))
            symPFunctionsIm = rand(Uniform(1.0,10.0), 1, (n+1))
            symPFunctions = symPFunctionsRe + symPFunctionsIm*im
        else # variable
            symPFunctions = Array{Number}(undef, 1,n+1)
            for i = 1:(n+1)
                # Each p_k is a polynomial function with random degree between 0 to 4 and random coefficients between 0 and 10
                coeffsNo = rand(1:5)
                pFunctionCoeffsRe = rand(Uniform(1.0,10.0), 1, coeffsNo)
                pFunctionCoeffsIm = rand(Uniform(1.0,10.0), 1,  coeffsNo)
                pFunctionCoeffs = pFunctionCoeffsRe + pFunctionCoeffsIm*im
                symPFunction = sum([pFunctionCoeffs[i+1]*t^(length(pFunctionCoeffs)-1-i) for i in 0:(length(pFunctionCoeffs)-1)])
                symPFunctions[i] = symPFunction
            end
        end
    end
    return symPFunctions
end 

generate_symPFunctions (generic function with 1 method)

## `generate_pFunctions`

In [37]:
function generate_pFunctions(n; random = false, constant = false)
    if random
        pFunctions = Array{Union{Function, Number}}(undef,1,n+1)
        pDerivMatrix = Array{Union{Function, Number}}(undef, n,n)
        for i = 1:(n+1)
            seed = rand(0:1)
            if seed == 0 # constant
                pFunctionRe = rand(Uniform(1.0,10.0), 1, 1)[1]
                pFunctionIm = rand(Uniform(1.0,10.0), 1, 1)[1]
                pFunction = pFunctionRe + pFunctionIm*im
                if i < n+1
                    pDerivMatrix[i,1] = pFunction
                    pDerivMatrix[i:i, 2:n] = zeros(1, n-1)
                end
            else # variable
                coeffsNo = rand(1:5)
                pFunctionCoeffsRe = rand(Uniform(1.0,10.0), 1, coeffsNo)
                pFunctionCoeffsIm = rand(Uniform(1.0,10.0), 1, coeffsNo)
                pFunctionCoeffs = pFunctionCoeffsRe + pFunctionCoeffsIm*im
                pFunction = get_polynomial(pFunctionCoeffs)
                if i < n+1
                    pDerivMatrix[i:i, 1:n] = [get_polynomialDeriv(pFunctionCoeffs, k) for k = 0:(n-1)]
                end
            end
            pFunctions[i] = pFunction
        end
    else
        if constant # constant
            pFunctionsRe = rand(Uniform(1.0,10.0), 1, (n+1))
            pFunctionsIm = rand(Uniform(1.0,10.0), 1, (n+1))
            pFunctions = pFunctionsRe + pFunctionsIm*im
            pDerivMatrix = complex(Matrix{Float64}(LinearAlgebra.I, n, n))
            for i = 1:n
                for j = 1:n
                    if j == 1
                        pDerivMatrix[i,j] = pFunctions[i]
                    else
                        pDerivMatrix[i,j] = 0
                    end
                end
            end
        else # variable
            pFunctions = Array{Union{Function, Number}}(undef,1,n+1)
            pDerivMatrix = Array{Union{Function, Number}}(undef,n,n)
            for i = 1:(n+1)
                # Each p_k is a polynomial function with random degree between 0 to 4 and random coefficients between 0 and 10
                coeffsNo = rand(1:5)
                pFunctionCoeffsRe = rand(Uniform(1.0,10.0), 1, coeffsNo)
                pFunctionCoeffsIm = rand(Uniform(1.0,10.0), 1, coeffsNo)
                pFunctionCoeffs = pFunctionCoeffsRe + pFunctionCoeffsIm*im
                if i < n+1
                    pDerivMatrix[i:i, 1:n] = [get_polynomialDeriv(pFunctionCoeffs, k) for k = 0:(n-1)]
                end
                pFunction = get_polynomial(pFunctionCoeffs)
                pFunctions[i] = pFunction
            end
        end
    end
    return pFunctions, pDerivMatrix
end

generate_pFunctions (generic function with 1 method)

## `generate_interval`

In [47]:
function generate_interval(n, a = 0, b = 1)
    if n == 1 || n == 2
        n = 3
    end
    array = Array{Any}(undef, 1, n)
    t = a
    for j=2:(n-1)
        array[j] = rand(Uniform(t, b))
        t = array[j]
    end
    array[1] = a
    array[n] = b
    interval = ntuple(i -> array[i], n)
    return interval
end

generate_interval (generic function with 3 methods)

In [39]:
function generate_fake_interval_1(n, a = 0, b = 1)
    if n == 1 || n == 2
        n = 4
    end
    array = Array{Any}(undef, 1, n)
    t = 0
    for j=2:(n-1)
        array[j] = rand(Uniform(t, b))
        t = array[j]
    end
    array[1] = a
    array[n] = "str"
    interval = ntuple(i -> array[i], n)
    return interval
end

generate_fake_interval_1 (generic function with 3 methods)

In [40]:
function generate_fake_interval_2(n, a = 0, b = 1)
    if n == 1 || n == 2
        n = 4
    end
    array = Array{Any}(undef, 1, n)
    t = 0
    for j=2:(n-1)
        array[n-j+1] = rand(Uniform(t, b))
        t = array[n-j+1]
    end
    array[1] = a
    array[n] = b
    interval = ntuple(i -> array[i], n)
    return interval
end

generate_fake_interval_2 (generic function with 3 methods)

## `generate_pFunctionsAndSymPFunctions`

In [41]:
function generate_pFunctionsAndSymPFunctions(n; random = false, constant = false)
    global t = symbols("t")
    interval = generate_interval(n,)
    if random
        pFunctions = Array{Union{Function, Number}}(undef,1,n+1)
        symPFunctions = Array{Number}(undef, 1,n+1)
        pDerivMatrix = Array{Union{Function, Number}}(undef,n,n)
        for i = 1:(n+1)
            seed = rand(0:1)
            if seed == 0 # constant
                pFunctionRe = rand(Uniform(1.0,10.0), 1, 1)[1]
                pFunctionIm = rand(Uniform(1.0,10.0), 1, 1)[1]
                pFunction = pFunctionRe + pFunctionIm*im
                symPFunction = pFunction
                if i < n+1
                    pDerivMatrix[i,1] = pFunction
                    pDerivMatrix[i:i, 2:n] = zeros(1, n-1)
                end
            else # variable
                coeffsNo = rand(1:5)
                pFunctionCoeffsRe = rand(Uniform(1.0,10.0), 1, coeffsNo)
                pFunctionCoeffsIm = rand(Uniform(1.0,10.0), 1, coeffsNo)
                pFunctionCoeffs = pFunctionCoeffsRe + pFunctionCoeffsIm*im
                if i < n+1
                    pDerivMatrix[i:i, 1:n] = [get_polynomialDeriv(pFunctionCoeffs, k) for k = 0:(n-1)]
                end
                pFunction = get_polynomial(pFunctionCoeffs)
                symPFunction = sum([pFunctionCoeffs[i+1]*t^(length(pFunctionCoeffs)-1-i) for i in 0:(length(pFunctionCoeffs)-1)])
            end
            pFunctions[i] = pFunction
            symPFunctions[i] = symPFunction
        end
    else
        if constant # constant
            pFunctionsRe = rand(Uniform(1.0,10.0), 1, (n+1))
            pFunctionsIm = rand(Uniform(1.0,10.0), 1, (n+1))
            pFunctions = pFunctionsRe + pFunctionsIm*im
            symPFunctions = pFunctions
            pDerivMatrix = Array{Union{Function, Number}}(undef, n, n)
            for i = 1:n
                for j = 1:n
                    if j == 1
                        pDerivMatrix[i,j] = pFunctions[i]
                    else
                        pDerivMatrix[i,j] = 0
                    end
                end
            end
        else # variable
            t = symbols("t")
            pFunctions = Array{Function}(undef, 1, n+1)
            symPFunctions = Array{Number}(undef, 1, n+1)
            pDerivMatrix = Array{Union{Function, Number}}(undef, n,n)
            for i = 1:(n+1)
                # Each p_k is a polynomial function with random degree between 0 to 4 and random coefficients between 0 and 10
                coeffsNo = rand(1:5)
                pFunctionCoeffsRe = rand(Uniform(1.0,10.0), 1, coeffsNo)
                pFunctionCoeffsIm = rand(Uniform(1.0,10.0), 1, coeffsNo)
                pFunctionCoeffs = pFunctionCoeffsRe + pFunctionCoeffsIm*im
                if i < n+1
                    pDerivMatrix[i:i, 1:n] = [get_polynomialDeriv(pFunctionCoeffs, k) for k = 0:(n-1)]
                end
                pFunction = get_polynomial(pFunctionCoeffs)
                pFunctions[i] = pFunction
                symPFunction = sum([pFunctionCoeffs[i+1]*t^(length(pFunctionCoeffs)-1-i) for i in 0:(length(pFunctionCoeffs)-1)])
                symPFunctions[i] = symPFunction
            end
        end
    end
    
    return pFunctions, symPFunctions, pDerivMatrix, interval
end

generate_pFunctionsAndSymPFunctions (generic function with 1 method)

## `rank_def`
Generates rank-deficient matrices

In [42]:
function rank_def(n)
    U = rand(Uniform(1.0,10.0), n, n)
    V = rand(Uniform(1.0,10.0), n, n)
    Fu = LinearAlgebra.qr(U)
    Fv = LinearAlgebra.qr(V)
    
    diags = rand(Uniform(1.0,10.0), 1, n-2)
    diag_vals = Array{Number}(undef, 1, n)
    for i=1:(n-2)
        diag_vals[i] = diags[i]
    end
    diag_vals[n], diag_vals[n-1] = 0, 0
    S = zeros(n,n)
    for i=1:n
        S[i,i] = diag_vals[i]
    end
    return Fu.Q*S*Fv.Q
end

rank_def (generic function with 1 method)

# Tests

## `test_generate_adjoint`

In [43]:
# Test the algorithm to generate valid adjoint U+
function test_generate_adjoint(n, k)
    global results = [true]
    global t = symbols("t")

    for counter = 1:k
        println("Test $counter")
        println("Testing the algorithm to generate valid adjoint U+: Constant p_k")
        (pFunctions, symPFunctions, pDerivMatrix, interval) = generate_pFunctionsAndSymPFunctions(n; random = false, constant = true)
        symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
        L = LinearDifferentialOperator(pFunctions, interval, symL)
        m = length(interval) - 1
        M = Array{Any}(undef, 1, m)
        N = Array{Any}(undef, 1, m)
        for i=1:m
            MCandRe = rand(Uniform(1.0,10.0), n, n)
            MCandIm = rand(Uniform(1.0,10.0), n, n)
            MCand = MCandRe + MCandIm*im
            NCandRe = rand(Uniform(1.0,10.0), n, n)
            NCandIm = rand(Uniform(1.0,10.0), n, n)
            NCand = NCandRe + NCandIm*im
            M[i] = MCand
            N[i] = NCand
        end
        U = VectorMultiBoundaryForm(M, N)
        println("Testing: order of L = $n")
        passed = false
        try
            adjoint = get_adjointU(L, U, pDerivMatrix)
            passed = true
            append!(results, passed)
        catch err
            println("Failed with $err")
        end
        if passed
            println("Passed!")
        end

        println("Testing the algorithm to generate valid adjoint U+: Variable p_k")
        # Generate variable p_k
        (pFunctions, symPFunctions, pDerivMatrix, interval) = generate_pFunctionsAndSymPFunctions(n; random = false, constant = false)
        symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
        L = LinearDifferentialOperator(pFunctions, interval, symL)
        m = length(interval) - 1
        M = Array{Any}(undef, 1, m)
        N = Array{Any}(undef, 1, m)
        for i=1:m
            MCandRe = rand(Uniform(1.0,10.0), n, n)
            MCandIm = rand(Uniform(1.0,10.0), n, n)
            MCand = MCandRe + MCandIm*im
            NCandRe = rand(Uniform(1.0,10.0), n, n)
            NCandIm = rand(Uniform(1.0,10.0), n, n)
            NCand = NCandRe + NCandIm*im
            M[i] = MCand
            N[i] = NCand
        end
        U = VectorMultiBoundaryForm(M, N)
        println("Testing: order of L = $n")
        try
            adjoint = get_adjointU(L, U, pDerivMatrix)
            passed = true
            append!(results, passed)
        catch err
            println("Failed with $err")
        end
        if passed
            println("Passed!")
        end

        println("Testing the algorithm to generate valid adjoint U+: Constant or variable p_k")
        # Generate p_k
        (pFunctions, symPFunctions, pDerivMatrix, interval) = generate_pFunctionsAndSymPFunctions(n; random = true)
        symL = SymLinearDifferentialOperator(symPFunctions, interval, t)
        L = LinearDifferentialOperator(pFunctions, interval, symL)
        m = length(interval) - 1
        M = Array{Any}(undef, 1, m)
        N = Array{Any}(undef, 1, m)
        for i=1:m
            MCandRe = rand(Uniform(1.0,10.0), n, n)
            MCandIm = rand(Uniform(1.0,10.0), n, n)
            MCand = MCandRe + MCandIm*im
            NCandRe = rand(Uniform(1.0,10.0), n, n)
            NCandIm = rand(Uniform(1.0,10.0), n, n)
            NCand = NCandRe + NCandIm*im
            M[i] = MCand
            N[i] = NCand
        end
        U = VectorMultiBoundaryForm(M, N)
        println("Testing: order of L = $n")
        try
            adjoint = get_adjointU(L, U, pDerivMatrix)
            passed = true
            append!(results, passed)
        catch err
            println("Failed with $err")
        end
        if passed
            println("Passed!")
        end
    end

    return all(results)
end

test_generate_adjoint (generic function with 1 method)

In [44]:
for n = 1:10
    println(test_generate_adjoint(n, 10))
end

Test 1
Testing the algorithm to generate valid adjoint U+: Constant p_k
Testing: order of L = 1
Passed!
Testing the algorithm to generate valid adjoint U+: Variable p_k
Testing: order of L = 1
Passed!
Testing the algorithm to generate valid adjoint U+: Constant or variable p_k
Testing: order of L = 1
Passed!
Test 2
Testing the algorithm to generate valid adjoint U+: Constant p_k
Testing: order of L = 1
Passed!
Testing the algorithm to generate valid adjoint U+: Variable p_k
Testing: order of L = 1
Passed!
Testing the algorithm to generate valid adjoint U+: Constant or variable p_k
Testing: order of L = 1
Passed!
Test 3
Testing the algorithm to generate valid adjoint U+: Constant p_k
Testing: order of L = 1
Passed!
Testing the algorithm to generate valid adjoint U+: Variable p_k
Testing: order of L = 1
Passed!
Testing the algorithm to generate valid adjoint U+: Constant or variable p_k
Testing: order of L = 1
Passed!
Test 4
Testing the algorithm to generate valid adjoint U+: Constant p_

Testing: order of L = 3
Passed!
Testing the algorithm to generate valid adjoint U+: Constant or variable p_k
Testing: order of L = 3
Passed!
Test 8
Testing the algorithm to generate valid adjoint U+: Constant p_k
Testing: order of L = 3
Passed!
Testing the algorithm to generate valid adjoint U+: Variable p_k
Testing: order of L = 3
Passed!
Testing the algorithm to generate valid adjoint U+: Constant or variable p_k
Testing: order of L = 3
Passed!
Test 9
Testing the algorithm to generate valid adjoint U+: Constant p_k
Testing: order of L = 3
Passed!
Testing the algorithm to generate valid adjoint U+: Variable p_k
Testing: order of L = 3
Passed!
Testing the algorithm to generate valid adjoint U+: Constant or variable p_k
Testing: order of L = 3
Passed!
Test 10
Testing the algorithm to generate valid adjoint U+: Constant p_k
Testing: order of L = 3
Passed!
Testing the algorithm to generate valid adjoint U+: Variable p_k
Testing: order of L = 3
Passed!
Testing the algorithm to generate val

InterruptException: InterruptException:

## `test_symLinearDifferentialOperatorDef`

In [48]:
# Test the SymLinearDifferentialOperator definition
function test_symLinearDifferentialOperatorDef(n, k)
    global results = [true]
    global t = symbols("t")

    for counter = 1:k
        println("Test $counter")
        println("Testing definition of SymLinearDifferentialOperator: symP_k are Function")
        symPFunctions = generate_symPFunctions(n; random = false, constant = false)
        interval_1 = generate_interval(n)
        passed = false
        try
            SymLinearDifferentialOperator(symPFunctions, interval_1, t)
            passed = true
        catch err
            println("Failed with $err")
        end
        if passed
            println("Passed!")
        end
        append!(results, passed)

        println("Testing definition of SymLinearDifferentialOperator: symP_k are constant")
        symPFunctions = generate_symPFunctions(n; random = false, constant = true)
        interval_2 = generate_interval(n)
        passed = false
        try
            SymLinearDifferentialOperator(symPFunctions, interval_2, t)
            passed = true
        catch err
            println("Failed with $err")
        end
        if passed
            println("Passed!")
        end
        append!(results, passed)

        println("Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number")
        symPFunctions = generate_symPFunctions(n; random = true)
        interval_3 = generate_interval(n)
        passed = false
        try
            SymLinearDifferentialOperator(symPFunctions, interval_3, t)
            passed = true
        catch err
            println("Failed with $err")
        end
        if passed
            println("Passed!")
        end
        append!(results, passed)

        println("Testing StructDefinitionError: symP_k should be SymPy.Sym or Number")
        symPFunctions = hcat(generate_symPFunctions(n-1; random = true), ["str"])
        interval_4 = generate_interval(n)
        passed = false
        try
            SymLinearDifferentialOperator(symPFunctions, interval_4, t)
        catch err
            if isa(err,StructDefinitionError) && err.msg == "symP_k should be SymPy.Sym or Number"
                passed = true
                println("Passed!")
            else
                println("Failed with $err")
            end
        end
        if !passed
            println("Failed!")
        end
        append!(results, passed)

        println("Testing StructDefinitionError: Only one free symbol is allowed in symP_k")
        interval_5 = generate_interval(n)
        r = symbols("r")
        passed = false
        try
            SymLinearDifferentialOperator([t+1 t+1 r*t+1], interval_5, t)
        catch err
            if isa(err,StructDefinitionError) && err.msg == "Only one free symbol is allowed in symP_k"
                passed = true
                println("Passed!")
            else
                println("Failed with $err")
            end
        end
        if !passed
            println("Failed!")
        end
        append!(results, passed)
            
        println("Testing StructDefinitionError: Interval must consist of numbers")
        symPFunctions = generate_symPFunctions(n; random = false, constant = false)
        interval_6 = generate_fake_interval_1(n)
        passed = false
        try
            SymLinearDifferentialOperator(symPFunctions, interval_6, t)
        catch err
            if isa(err,StructDefinitionError) && err.msg == "Interval must consist of numbers"
                passed = true
                println("Passed!")
            else
                println("Failed with $err")
            end
        end
        if !passed
            println("Failed!")
        end
        append!(results, passed)
    
        println("Testing StructDefinitionError: Terms in interval must be strictly increasing")
        symPFunctions = generate_symPFunctions(n; random = false, constant = false)
        interval_7 = generate_fake_interval_2(n)
        passed = false
        try
            SymLinearDifferentialOperator(symPFunctions, interval_7, t)
        catch err
            if isa(err,StructDefinitionError) && err.msg == "Terms in interval must be strictly increasing"
                passed = true
                println("Passed!")
            else
                println("Failed with $err")
            end
        end
        if !passed
            println("Failed!")
        end
        append!(results, passed)
    end
    return all(results)
end

test_symLinearDifferentialOperatorDef (generic function with 1 method)

In [49]:

    println(test_symLinearDifferentialOperatorDef(n, 10))
end

Test 1
Testing definition of SymLinearDifferentialOperator: symP_k are Function
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are constant
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Testing StructDefinitionError: Interval must consist of numbers
Passed!
Testing StructDefinitionError: Terms in interval must be strictly increasing
Passed!
Test 2
Testing definition of SymLinearDifferentialOperator: symP_k are Function
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are constant
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Testi

Passed!
Test 7
Testing definition of SymLinearDifferentialOperator: symP_k are Function
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are constant
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Testing StructDefinitionError: Interval must consist of numbers
Passed!
Testing StructDefinitionError: Terms in interval must be strictly increasing
Passed!
Test 8
Testing definition of SymLinearDifferentialOperator: symP_k are Function
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are constant
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passe

Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Testing StructDefinitionError: Interval must consist of numbers
Passed!
Testing StructDefinitionError: Terms in interval must be strictly increasing
Passed!
Test 2
Testing definition of SymLinearDifferentialOperator: symP_k are Function
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are constant
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Testing StructDefinitionError: Interval must consist of numbers
Passed!
Testing StructDefinitionError: Terms in interval must be strictly increasing
Passed!
Test 3
Testing definition of SymLinearDifferentialOperator: symP_k are Function
Passed!
Testing defini

Passed!
Test 6
Testing definition of SymLinearDifferentialOperator: symP_k are Function
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are constant
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Testing StructDefinitionError: Interval must consist of numbers
Passed!
Testing StructDefinitionError: Terms in interval must be strictly increasing
Passed!
Test 7
Testing definition of SymLinearDifferentialOperator: symP_k are Function
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are constant
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passe

Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Testing StructDefinitionError: Interval must consist of numbers
Passed!
Testing StructDefinitionError: Terms in interval must be strictly increasing
Passed!
true
Test 1
Testing definition of SymLinearDifferentialOperator: symP_k are Function
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are constant
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Testing StructDefinitionError: Interval must consist of numbers
Passed!
Testing StructDefinitionError: Terms in interval must be strictly increasing
Passed!
Test 2
Testing definition of SymLinearDifferentialOperator: symP_k are Function
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are constant
Passed!
Test

Passed!
Testing StructDefinitionError: Terms in interval must be strictly increasing
Passed!
Test 5
Testing definition of SymLinearDifferentialOperator: symP_k are Function
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are constant
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Testing StructDefinitionError: Interval must consist of numbers
Passed!
Testing StructDefinitionError: Terms in interval must be strictly increasing
Passed!
Test 6
Testing definition of SymLinearDifferentialOperator: symP_k are Function
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are constant
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
P

Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are constant
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Testing StructDefinitionError: Interval must consist of numbers
Passed!
Testing StructDefinitionError: Terms in interval must be strictly increasing
Passed!
Test 10
Testing definition of SymLinearDifferentialOperator: symP_k are Function
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are constant
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Testing StructDefinitionError: Interval must consist of numbers
Passed!
Testing Stru

In [60]:
println(test_symLinearDifferentialOperatorDef(3, 10))

Test 1
Testing definition of SymLinearDifferentialOperator: symP_k are Function
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are constant
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Testing StructDefinitionError: Interval must consist of numbers
Passed!
Testing StructDefinitionError: Terms in interval must be strictly increasing
Failed!
Test 2
Testing definition of SymLinearDifferentialOperator: symP_k are Function
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are constant
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!
Testi

## `test_LinearDifferentialOperatorDef`

In [61]:
# Test the LinearDifferentialOperator definition
function test_LinearDifferentialOperatorDef(n, k)
    global results = [true]
    global t = symbols("t")
    
    for counter = 1:k
        println("Test $counter")

        # Variable p_k
        println("Testing definition of LinearDifferentialOperator: p_k are Function")
        (pFunctions, symPFunctions, pDerivMatrix, interval_1) = generate_pFunctionsAndSymPFunctions(n; random = false, constant = false)
        symL = SymLinearDifferentialOperator(symPFunctions, interval_1, t)
        passed = false
        try
            L = LinearDifferentialOperator(pFunctions, interval_1, symL)
            passed = true
        catch err
            println("Failed with $err")
        end
        if passed
            println("Passed!")
        end
        append!(results, passed)

        # Constant coefficients
        println("Testing definition of LinearDifferentialOperator: p_k are Constants")
        (pFunctions, symPFunctions, pDerivMatrix, interval_2) = generate_pFunctionsAndSymPFunctions(n; random = false, constant = true)
        symL = SymLinearDifferentialOperator(symPFunctions, interval_2, t)
        passed = false
        try
            LinearDifferentialOperator(pFunctions, interval_2, symL)
            passed = true
        catch err
            println("Failed with $err")
        end
        if passed
            println("Passed!")
        end
        append!(results, passed)

        # Mixed coefficients
        println("Testing definition of LinearDifferentialOperator: p_k are mixed")
        # (pFunctions, symPFunctions, pDerivMatrix) = generate_pFunctionsAndSymPFunctions(n; random = true)
        symL = SymLinearDifferentialOperator([1 1 t+1], interval_2, t)
        passed = false
        try
            LinearDifferentialOperator([1 t->1 t->t+1], interval_2, symL)
            passed = true
        catch err
            println("Failed with $err")
        end
        if passed
            println("Passed!")
        end
        append!(results, passed)

        println("Testing StructDefinitionError: p_k should be Function or Number")
        pFunctions = hcat(generate_pFunctions(n-1; random = true)[1], ["str"])
        passed = false
        try
            LinearDifferentialOperator(['s' 1 1], interval_2, symL)
        catch err
            if err.msg == "p_k should be Function or Number" && (isa(err,StructDefinitionError))
                passed = true
                println("Passed!")
            else
                println("Failed with $err")
            end
        end
        if !passed
            println("Failed!")
        end
        append!(results, passed)

        println("Testing StructDefinitionError: Number of p_k and symP_k do not match")
        symL = SymLinearDifferentialOperator([1 1 t+1], interval_2, t)
        passed = false
        try
            LinearDifferentialOperator([1 t->1], interval_2, symL)
        catch err
            if err.msg == "Number of p_k and symP_k do not match" && (isa(err, StructDefinitionError))
                passed = true
                println("Passed!")
            else
                println("Failed with $err")
            end
        end
        if !passed
            println("Failed!")
        end
        append!(results, passed)

        println("Testing StructDefinitionError: Intervals of L and symL do not match")
        symL = SymLinearDifferentialOperator([1 1 t+1], interval_2, t)
        interval_3 = generate_interval(n)
        passed = false
        try
            LinearDifferentialOperator([1 t->1 t->t+1], interval_3, symL)
        catch err
            if err.msg == "Intervals of L and symL do not match" && (isa(err, StructDefinitionError))
                passed = true
                println("Passed!")
            else
                println("Failed with $err")
            end
        end
        if !passed
            println("Failed!")
        end
        append!(results, passed)
    
    end
    return all(results)
end

test_LinearDifferentialOperatorDef (generic function with 1 method)

In [ ]:
for n = 1:10
    println(test_LinearDifferentialOperatorDef(n, 10))
end

Test 1
Testing definition of LinearDifferentialOperator: p_k are Function
Passed!
Testing definition of LinearDifferentialOperator: p_k are Constants
Passed!
Testing definition of LinearDifferentialOperator: p_k are mixed
Passed!
Testing StructDefinitionError: p_k should be Function or Number
Passed!
Testing StructDefinitionError: Number of p_k and symP_k do not match
Passed!
Testing StructDefinitionError: Intervals of L and symL do not match
Passed!
Test 2
Testing definition of LinearDifferentialOperator: p_k are Function
Passed!
Testing definition of LinearDifferentialOperator: p_k are Constants
Passed!
Testing definition of LinearDifferentialOperator: p_k are mixed
Passed!
Testing StructDefinitionError: p_k should be Function or Number
Passed!
Testing StructDefinitionError: Number of p_k and symP_k do not match
Passed!
Testing StructDefinitionError: Intervals of L and symL do not match
Passed!
Test 3
Testing definition of LinearDifferentialOperator: p_k are Function
Failed with Inte

In [ ]:
3

## `test_vectorBoundaryFormDef`

In [854]:
# Test the VectorBoundaryForm definition
function test_VectorMultiBoundaryFormDef(n, k)
    global results = [true]
    if n == 1
        n = 2
    end
    for counter = 1:k
        println("Test $counter")
        
        println("Testing the definition of VectorBoundaryForm")
        interval_1 = generate_interval(n)
        m = length(interval_1) - 1
        M = Array{Any}(undef, 1, m)
        N = Array{Any}(undef, 1, m)
        for j=1:m
            MRe = rand(Uniform(1.0,10.0), n, n)
            MIm = rand(Uniform(1.0,10.0), n, n)
            M_j = MRe + MIm*im
            M[j] = M_j
            NRe = rand(Uniform(1.0,10.0), n, n)
            NIm = rand(Uniform(1.0,10.0), n, n)
            N_j = NRe + NIm*im
            N[j] = N_j
        end
        passed = false
        try
            VectorMultiBoundaryForm(M, N)
            passed = true
        catch err
            println("Failed with $err")
        end
        if passed
            println("Passed!")
        end
        append!(results, passed)

        println("Testing StructDefinitionError: Entries of M_i, N_i should be Number")
        interval_2 = generate_interval(n)
        m = length(interval_2) - 1
        M = Array{Any}(undef, 1, m)
        N = Array{Any}(undef, 1, m)
        for j=1:m
            MRe = rand(Uniform(1.0,10.0), n, n)
            MIm = rand(Uniform(1.0,10.0), n, n)
            M_j = MRe + MIm*im
            M_j = convert(Array{Any}, M_j)
            M_j[j,j] = "str"
            M[j] = M_j
            NRe = rand(Uniform(1.0,10.0), n, n)
            NIm = rand(Uniform(1.0,10.0), n, n)
            N_j = NRe + NIm*im
            N_j = convert(Array{Any}, N_j)
            N_j[j,j] = "str"
            N[j] = N_j
        end
        passed = false
        try
            VectorMultiBoundaryForm(M, N)
        catch err
            if err.msg == "Entries of M_i, N_i should be Number" && isa(err, StructDefinitionError)
                passed = true
                println("Passed!")
            else
                println("Failed with $err")
            end
        end
        if !passed
            println("Failed!")
        end
        append!(results, passed)

        println("Testing StructDefinitionError: M_i, N_i dimensions do not match")
        interval_3 = generate_interval(n)
        m = length(interval_3) - 1
        M = Array{Any}(undef, 1, m)
        N = Array{Any}(undef, 1, m)
        for j=1:m
            MRe = rand(Uniform(1.0,10.0), n, n-1)
            MIm = rand(Uniform(1.0,10.0), n, n-1)
            M_j = MRe + MIm*im
            M[j] = M_j
            NRe = rand(Uniform(1.0,10.0), n, n)
            NIm = rand(Uniform(1.0,10.0), n, n)
            N_j = NRe + NIm*im
            N[j] = N_j
        end
        passed = false
        try
            VectorMultiBoundaryForm(M, N)
        catch err
            if err.msg == "M_i, N_i dimensions do not match" && isa(err,StructDefinitionError)
                passed = true
                println("Passed!")
            else
                println("Failed with $err")
            end
        end
        if !passed
            println("Failed!")
        end
        append!(results, passed)

        println("Testing StructDefinitionError: M_i, N_i should be square matrices")
        interval_4 = generate_interval(n)
        m = length(interval_4) - 1
        M = Array{Any}(undef, 1, m)
        N = Array{Any}(undef, 1, m)
        for j=1:m
            MRe = rand(Uniform(1.0,10.0), n, n-1)
            MIm = rand(Uniform(1.0,10.0), n, n-1)
            M_j = MRe + MIm*im
            M[j] = M_j
            NRe = rand(Uniform(1.0,10.0), n, n-1)
            NIm = rand(Uniform(1.0,10.0), n, n-1)
            N_j = NRe + NIm*im
            N[j] = N_j
        end
        passed = false
        try
            VectorMultiBoundaryForm(M, N)
        catch err
            if err.msg == "M_i, N_i should be square matrices" && isa(err,StructDefinitionError)
                passed = true
                println("Passed!")
            else
                println("Failed with $err")
            end
        end
        if !passed
            println("Failed!")
        end
        append!(results, passed)

        println("Testing StructDefinitionError: Boundary operators not linearly independent")
        
        interval_5 = generate_interval(n)
        m = length(interval_5) - 1
        M = Array{Any}(undef, 1, m)
        N = Array{Any}(undef, 1, m)
        for j=1:m
            M[j] = rank_def(n)
            N[j] = M[j]
        end
        passed = false
        try
            VectorMultiBoundaryForm(M, N)
        catch err
            if err.msg == "Boundary operators not linearly independent" && isa(err,StructDefinitionError)
                passed = true
                    println("Passed!")
                else
                    println("Failed with $err")
                end
            end
        if !passed
            println("Failed!")
        end
        append!(results, passed)
    end

    return all(results)
end

test_VectorMultiBoundaryFormDef (generic function with 1 method)

In [842]:
for n = 1:10
    println(test_VectorMultiBoundaryFormDef(n, 10))
end

Test 1
Testing the definition of VectorBoundaryForm
Passed!
Testing StructDefinitionError: Entries of M_i, N_i should be Number


BoundsError: BoundsError: attempt to access 1×1 Array{Any,2} at index [2, 2]